In [ ]:
from iragca.matplotlib import Styles
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import torch
from torchvision import transforms

from lib.architectures import SimpleCNN
from lib.config import Directories
from lib.data import PlantDocDiseaseDetection

plt.style.use(Styles.ML.value)

model_name = "disease_detection_model"


IMAGE_SIZE = (32, 32)
transform_pipeline = transforms.Compose(
    [
        transforms.Resize(IMAGE_SIZE),
        transforms.ToTensor(),
    ]
)


dataset = PlantDocDiseaseDetection(data_path=Directories.EXTERNAL_DATA_DIR.value / "kagglehub" / "plantdoc", transforms=transform_pipeline)

dataset[0]

(tensor([[[0.3176, 0.2275, 0.3216,  ..., 0.5294, 0.4824, 0.5333],
          [0.3882, 0.4431, 0.3529,  ..., 0.5333, 0.5020, 0.5490],
          [0.4314, 0.4824, 0.2549,  ..., 0.5412, 0.5098, 0.5255],
          ...,
          [0.5176, 0.3804, 0.4745,  ..., 0.2471, 0.2510, 0.3686],
          [0.4902, 0.4118, 0.6510,  ..., 0.2471, 0.2745, 0.3529],
          [0.4941, 0.5725, 0.5765,  ..., 0.2627, 0.2588, 0.3294]],
 
         [[0.3020, 0.2431, 0.4196,  ..., 0.7569, 0.7059, 0.6824],
          [0.4039, 0.4863, 0.4549,  ..., 0.7529, 0.7294, 0.7216],
          [0.4706, 0.5255, 0.3020,  ..., 0.7490, 0.7294, 0.6706],
          ...,
          [0.5059, 0.3882, 0.4510,  ..., 0.4235, 0.4235, 0.5294],
          [0.4941, 0.3961, 0.5961,  ..., 0.4118, 0.4392, 0.5059],
          [0.4510, 0.5059, 0.4941,  ..., 0.4314, 0.4196, 0.4824]],
 
         [[0.1529, 0.2078, 0.1922,  ..., 0.5255, 0.4980, 0.5333],
          [0.1216, 0.1765, 0.1020,  ..., 0.4902, 0.4941, 0.5176],
          [0.1765, 0.2000, 0.1569,  ...,

In [7]:
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
plantdoc_loader = DataLoader(dataset, batch_size=32, shuffle=True)


model = SimpleCNN(channels=3, output_dim=1)
model.load_state_dict(torch.load(Directories.MODELS_DIR.value / f"{model_name}.pth"))


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
all_preds = []
all_labels = []
THRESHOLD = 0.5
with torch.no_grad():
    for images, labels in tqdm(plantdoc_loader, desc="Evaluating on PlantDoc Dataset"):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = (outputs >= THRESHOLD).long()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

f1 = f1_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, all_preds)

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

Evaluating on PlantDoc Dataset:   0%|          | 0/92 [00:00<?, ?it/s]

Evaluating on PlantDoc Dataset: 100%|██████████| 92/92 [01:04<00:00,  1.42it/s]

F1 Score: 0.8050
Accuracy: 0.7246
